# Spark assignment 1: Pairs

Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

    red_apple	100500
    crazy_zoo	42

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

Dataset location: /data/wiki/en_articles_part

The result on the sample dataset:

    narodnaya_gazeta    1
    narodnaya_volya 9

In [1]:
# Python2 on grader, Python3 in docker
import sys
if sys.version_info.major == 3:
    import os
    os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
    os.environ['PYTHONHASHSEED'] = '42'
    unicode = str

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

In [3]:
# for word in result[:5]:
#     print(word)

In [4]:
# Lowercase texts

lwiki = wiki.map(lambda text: [s.lower() for s in text])

In [5]:
# Generate all pairs

def word_list_to_pairs(ls):
    return zip(ls, ls[1:])

pairs = lwiki.flatMap(word_list_to_pairs)
# print(pairs.take(5))

In [6]:
# Continue only with "narodnaya"

nar_pairs = pairs.filter(lambda p: p[0] == 'narodnaya')

# print(nar_pairs.take(5))

In [7]:
# Pairs to keys

key_pairs = nar_pairs.map(lambda p: (p[0] + '_' + p[1], 1))
# print(key_pairs.take(5))

In [8]:
key_pairs.cache()

PythonRDD[3] at RDD at PythonRDD.scala:48

In [9]:
sumFunc = lambda x, y: x + y

counted = key_pairs.aggregateByKey(0, sumFunc, sumFunc)
counted.cache()

# print(counted.take(5))

PythonRDD[8] at RDD at PythonRDD.scala:48

In [10]:
# counted.lookup('narodnaya_volya')

In [11]:
sorted_result = counted.sortByKey()

for (k, v) in sorted_result.collect():
    print("%s\t%s" % (k, v))

narodnaya_gazeta	1
narodnaya_volya	9
